In [ ]:
from decodes.core import *
from decodes.io.jupyter_out import JupyterOut
import math

out = JupyterOut.unit_square( )

# The Raster Family
todo

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P22.jpg" style="width: 200px; display: inline;">


<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P30.jpg" style="width: 200px; display: inline;">


## Rasters

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.C05.jpg" style="width: 200px; display: inline;">

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P07.jpg" style="width: 200px; display: inline;">


In [1]:
"""
Raster Get and Set Methods
The translation from a given x,y coordinate of a pixel and its index in the 
_pixel collection may be obtained by a simple equation that accounts for the 
number of pixels per row of the Raster.
"""
def get(self,x,y):
    return self._pixels[y*self._dim[0]+x]

def set(self,x,y,value):
    self._pixels[y*self.px_width+x] = value

In [ ]:
"""
Raster Pixel Dimension Properties
A number of properties are defined to ease access to the pixel dimension of a 
Raster.
"""
@property
def px_dim(self):
    return self._dim
    
@property
def px_width(self):
    return int(self._dim[0])

@property
def px_height(self):
    return int(self._dim[1])
    
@property
def px_count(self):
    return self.px_width*self.px_height

In [ ]:
"""
Raster Pixel Addresses
Many processes require iteration over every available pixel in a Raster. Here, 
all the valid pixel addresses are returned as a single collection.
"""
@property
def addresses(self):
    return itertools.product(range(self.px_width),range(self.px_height))

In [ ]:
"""
Raster Population
Here, every pixel in a Raster is populated with a given value.
"""
def populate(self,val):
    self._pixels = [val]*self.px_count

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P31.jpg" style="width: 200px; display: inline;">


In [ ]:
"""
Raster Initialization
The initialization of a Raster proceeds by first setting the pixel dimension 
and accounting for any given keyword arguments before initializing the _pixels 
collection. Note that this collection is defined here, but contains no objects. 
Populating the pixels of a Raster is left to descendant types.
"""
class Raster(object):
    def __init__(self,pixel_dim=(20,20),**kwargs):
        self._dim = pixel_dim
        self.include_corners = False
        if "include_corners" in kwargs: 
            self.include_corners = kwargs["include_corners"]
        self.wrap = False
        if "wrap" in kwargs: 
            self.wrap = kwargs["wrap"]
        
        # the _pixels collection is initialized but not populated
        self._pixels = []

### Raster Descendants

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P24.jpg" style="width: 200px; display: inline;">


In [ ]:
"""
Initialization of Raster Descendants
The direct descendants of Raster present very similar initialization routines. 
Each calls the initialization of its parent before populating the _pixels 
collection with an appropriate object type. Optionally, as we seen in BoolField
and ValueField, the default values of keyword arguments may be adjusted.
"""
class BoolField(Raster):
    def __init__(self, pixel_dim=None, initial_value=False, **kwargs):
        if "wrap" not in kwargs: kwargs["wrap"] = True
        super(BoolField,self).__init__(pixel_dim,**kwargs)
        self.populate(initial_value)
        
class ValueField(Raster):
    def __init__(self, pixel_dim=None, initial_value=0.0, **kwargs):
        if "wrap" not in kwargs: kwargs["wrap"] = True
        super(ValueField,self).__init__(pixel_dim,**kwargs)
        self.populate(initial_value)

class Image(Raster):
    def __init__(self,pixel_dim,initial_color = Color(),**kwargs):
        super(Image,self).__init__(pixel_dim,**kwargs)
        self.populate(initial_color)

In [ ]:
"""
Unique Methods of a ValueField
Some descendants of Raster require methods that are only appropriate to their 
contained type. Here we see two methods that are unique to the ValueField type.
"""
@property
def max_value(self):
    return max(self._pixels)

@property
def min_value(self):
    return min(self._pixels)

## Grids

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P25.jpg" style="width: 200px; display: inline;">


### Bounds Objects in Decod.es

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/3.00.D88 Bounds Large.jpg" style="width: 800px; display: inline;">


In [ ]:
"""
Construction of a Decod.es Bounds
A 2d Bounds is constructed by two intervals, a 3d bounds by three. 
Alternatively, we may construct a Bounds by centerpoint and dimension.
"""

bnds_2d = Bounds(ival_x=Interval(-2,2), ival_y=Interval(-6,1))
bnds_3d = Bounds(ival_x=Interval(), ival_y=Interval(2,1), ival_z=Interval())
bnds_by_cpt = Bounds(center=Point(), dim_x=3.0, dim_y=1.5)

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P32.jpg" style="width: 200px; display: inline;">


In [ ]:
"""
Bounds Attributes
"""
print bnds_2d.cpt
print bnds_3d.dim_x
print bnds_by_cpt.ival_x

In [ ]:
"""
Bounds Encompass
"""
pts = [Point(-1,2), Point(-3,0), Point(2,2), Point(0,4)]
bnds = Bounds.encompass(pts)

### Grid Objects in Decod.es

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.C06.jpg" style="width: 200px; display: inline;">

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P33.jpg" style="width: 200px; display: inline;">


In [ ]:
"""
Grid Initialization
This abstract class for storing information in a spatialized raster grid format
s constructed first by calling the Raster initialization, and then establishing 
a two-dimensional spatial bounds _bnds. Finally, a number of values concerning 
the division of this spatial bounds are pre-calculated.
"""
class Grid(Raster):
    def __init__(self,pixel_dim=None,bnds=None,**kwargs):
        super(Grid,self).__init__(pixel_dim,**kwargs)
        if bnds is None: self._bnds = Bounds.unit_square()
        # enforces a two-dimensional Bounds
        else: self._bnds = Bounds(ival_x=bnds.ival_x,ival_y=bnds.ival_y ) 
        self._recalculate_base_pts()

In [ ]:
"""
Base Point Calculation
A Grid pre-calculates values that describe the center Point of each grid cell, 
as well as divisions of the overall x and y Intervals. This pre-calculation 
eases the burden on subsequent method calls, but must be re-calculated any time 
the stored _bnds is altered.
"""
def _recalculate_base_pts(self):
    self._base_pts = []
    self._ivals_x = self.bnds.ival_x//self.px_width
    self._ivals_y = self.bnds.ival_y//self.px_height
    for ival_y in self._ivals_y:
        for ival_x in self._ivals_x:
            self._base_pts.append(Point(ival_x.mid, ival_y.mid))

In [ ]:
"""
Gridcell Center Point
A method that returns the center point of the cell associated with the given 
address.
"""
def get_cpt(self,x,y):
    return self._base_pts[y*self.px_width+x]

In [ ]:
"""
Grid Address Near
A method that returns the address of the grid cells nearest the given location.
This method may be passed either a point or an x,y coordinate.
"""    
def address_near(self,a,b=None):
    pt = Point(a,b)
    # find index of interval containing x-coordinate
    if pt.x <= self.bnds.ival_x.a : idx_x = 0
    elif pt.x >= self.bnds.ival_x.b : idx_x = self.px_width - 1
    else: idx_x = [pt.x in ival for ival in self._ivals_x].index(True)
    # find index of interval containing y-coordinate
    if pt.y <= self.bnds.ival_y.a : idx_y = 0
    elif pt.y >= self.bnds.ival_y.b : idx_y = self.px_height - 1
    else: idx_y = [pt.y in ival for ival in self._ivals_y].index(True)
    # return address tuple
    return idx_x, idx_y

In [ ]:
"""
Grid Addresses Near
Here, a collection is returned containing addresses of grid cells near to the 
given location. This method may be passed either a point or an x,y coordinate.
"""         
def addresses_near(self,a,b=None):
    pt = Point(a,b)
    add = self.address_near(pt)
    dx = 1 if pt.x > self._ivals_x[add[0]].mid else -1
    dy = 1 if pt.y > self._ivals_y[add[1]].mid else -1
    adds = [add,(add[0]+dx,add[1]),(add[0]+dx,add[1]+dy),(add[0],add[1]+dy)]
    adds = filter(lambda add: add[0]>=0 and add[0]<self.px_width, adds)
    adds = filter(lambda add: add[1]>=0 and add[1]<self.px_height, adds)
    return sorted(adds)

In [ ]:
"""
Grid Center Points Near
The center points of cells near the given location are returned by this method. 
Note the use of the splat operator to pack the pixel address tuple so that they
are passed as separate arugments to the get_cpt method.
"""       
def cpts_near(self,a,b=None):
    return [self.get_cpt(*add) for add in self.addresses_near(a,b)]

In [ ]:
"""
Vec and Vecs Near
Two methods that return closest vector or the closest set of vectors to the 
given location. Again, the splat operator is employed for argument packing.
"""
def vec_near(self,a,b=None):
    return self.get(*self.address_near(a,b))

def vecs_near(self,a,b=None):
    return [self.get(*add) for add in self.addresses_near(a,b)]

In [ ]:
"""
Average Vec Near
Returns an average vector from the near vectors around the given location. 
"""
def avg_vec_near(self,sample_pt):
    vecs = self.vecs_near(sample_pt)
    dists = [1.0/sample_pt.distance2(pt) for pt in self.cpts_near(sample_pt)]
    tot = sum(dists)
    weights = [dist/tot for dist in dists]
    
    vec = Vec()
    for v,w in zip(vec,weights): vec += v*w
    return vec
    

## Multiple Inheritance

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P26.jpg" style="width: 200px; display: inline;">


<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P10.jpg" style="width: 200px; display: inline;">


In [ ]:
"""
The Multiple Inheritance of a ValueGrid
In one simple statement we define the ValueGrid type, which inherits the 
spatialized raster structure of a Grid and the methods related to storing 
numeric values of a ValueField
"""
class ValueGrid(Grid, ValueField):

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P11.jpg" style="width: 200px; display: inline;">


In [ ]:
"""
Point-Value Quartet
A quartet represents the center points and values of four ValueGrid cells that 
meet at a corner. Since four cells are required, the effective pixel_dim of 
quartets is one fewer in each dimension than that of the ValueField which they 
describe.
"""
def quartet(self,x,y):
    idxs = [(x,y),(x+1,y),(x+1,y+1),(x,y+1)]
    # return a Tuple of (values, pts)
    return [self.get(*xy) for xy in idxs], [self.get_cpt(*xy) for xy in idxs]